In [1]:
from IPython import display

In [2]:
try:
    import nbformat
except:
    !pip install nbformat
display.clear_output()

In [3]:
%run /content/drive/MyDrive/house_predict/process/test_31102025/model/train.ipynb

+ action: train_feat01
+ feat_path: /content/drive/MyDrive/house_predict/process/exps/trainbase_31102025/data01.npz
+ seed: 42
+ exp_dir: /content/drive/MyDrive/house_predict/process/exps
+ exp_name: trainbase_31102025
+ data_dir: /content/drive/MyDrive/house_predict/data
+ verbose: True
+ save_dir: /content/drive/MyDrive/house_predict/process/exps/trainbase_31102025
---------- information ----------
train-col: {'Utilities', 'BsmtFinSF1', 'SaleCondition', 'BsmtExposure', 'Alley', 'LowQualFinSF', 'KitchenAbvGr', 'MiscVal', 'BsmtFinSF2', 'GarageYrBlt', 'WoodDeckSF', 'GarageFinish', 'BsmtFullBath', 'MiscFeature', '2ndFlrSF', 'MasVnrArea', 'CentralAir', 'BsmtCond', 'TotalBsmtSF', 'RoofMatl', 'LandContour', '1stFlrSF', 'MasVnrType', 'Heating', 'GrLivArea', 'BsmtFinType2', 'Fireplaces', 'BsmtUnfSF', 'FireplaceQu', 'Fence', '3SsnPorch', 'FullBath', 'HeatingQC', 'ExterCond', 'HalfBath', 'MoSold', 'OverallQual', 'TotRmsAbvGrd', 'BsmtFinType1', 'LandSlope', 'MSSubClass', 'YearRemodAdd', 'Overall

# Import lib

In [4]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import os

data = np.load("/content/drive/MyDrive/house_predict/process/exps/trainbase_31102025/data01.npz", allow_pickle=True)

In [5]:
with np.load('/content/drive/MyDrive/house_predict/process/exps/trainbase_31102025/data01.npz') as z:
    print(z.files)

['train_data', 'test_data', 'train_columns', 'test_columns', 'allow_pickle']


In [6]:
df_train = pd.DataFrame(data['train_data'], columns=data['train_columns'])
df_test = pd.DataFrame(data['test_data'], columns=data['test_columns'])

In [7]:
df_train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,AreaPerRoom,Loc_Type_Style_Cond,GarageUtilization,AmenityScore,Living_to_LotRatio,QualMinusCond,LivableArea,Bsmt_to_LivArea,Loc_Lot,SoldSinceRemod
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,190.0,CollgCr_1Fam_2Story_Normal,182.666667,6,0.202343,2,3114,0.500292,CollgCr_0,5
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,180.285714,Veenker_1Fam_1Story_Normal,153.333333,8,0.131445,-2,2984,0.999208,Veenker_0,31
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,255.142857,CollgCr_1Fam_2Story_Normal,202.666667,8,0.158741,2,3314,0.514829,CollgCr_0,6
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,214.625,Crawfor_1Fam_2Story_Abnorml,160.5,5,0.179772,2,3115,0.440047,Crawfor_0,36
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,219.8,NoRidge_1Fam_2Story_Normal,209.0,8,0.154127,3,4179,0.520691,NoRidge_0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,...,205.875,Gilbert_1Fam_2Story_Normal,153.333333,8,0.208007,1,3060,0.578277,Gilbert_0,7
1456,1457,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,...,259.125,NWAmes_1Fam_1Story_Normal,166.666667,10,0.157332,0,4115,0.743491,NWAmes_0,22
1457,1458,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,...,234.0,Crawfor_1Fam_2Story_Normal,126.0,10,0.258764,-2,3744,0.492097,Crawfor_0,4
1458,1459,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,...,179.666667,NAmes_1Fam_1Story_Normal,120.0,3,0.110928,-1,2396,0.999073,NAmes_0,14


In [8]:
df_test

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,AreaPerRoom,Loc_Type_Style_Cond,GarageUtilization,AmenityScore,Living_to_LotRatio,QualMinusCond,LivableArea,Bsmt_to_LivArea,Loc_Lot,SoldSinceRemod
0,1461,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,Inside,...,149.333333,NAmes_1Fam_1Story_Normal,365.0,3,0.077089,-1,2508.0,0.983278,NAmes_0,49
1,1462,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,Corner,...,189.857143,NAmes_1Fam_1Story_Normal,156.0,3,0.093146,0,2970.0,0.999248,NAmes_1,52
2,1463,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,Inside,...,232.714286,Gilbert_1Fam_2Story_Normal,160.666667,8,0.117779,0,3039.0,0.569325,Gilbert_1,12
3,1464,60,RL,78.0,9978,Pave,IR1,Lvl,AllPub,Inside,...,200.5,Gilbert_1Fam_2Story_Normal,156.666667,8,0.160738,0,3000.0,0.576947,Gilbert_0,12
4,1465,120,RL,43.0,5005,Pave,IR1,HLS,AllPub,Inside,...,213.333333,StoneBr_TwnhsE_1Story_Normal,168.666667,6,0.255693,3,3066.0,0.999219,StoneBr_0,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,Reg,Lvl,AllPub,Inside,...,182.0,MeadowV_Twnhs_2Story_Normal,0.0,3,0.563758,-3,1638.0,0.499543,MeadowV_0,36
1455,2916,160,RM,21.0,1894,Pave,Reg,Lvl,AllPub,Inside,...,156.0,MeadowV_TwnhsE_2Story_Abnorml,143.0,3,0.576253,-1,1924.0,0.499543,MeadowV_0,36
1456,2917,20,RL,160.0,20000,Pave,Reg,Lvl,AllPub,Inside,...,153.0,Mitchel_1Fam_1Story_Abnorml,192.0,5,0.061197,-2,3024.0,0.999184,Mitchel_1,10
1457,2918,85,RL,62.0,10441,Pave,Reg,Lvl,AllPub,Inside,...,138.571429,Mitchel_1Fam_SFoyer_Normal,0.0,3,0.092894,0,1882.0,0.939238,Mitchel_0,14


In [9]:
df_test.isnull().sum()[df_test.isnull().sum() > 0]

,0


In [10]:
df_train.isnull().sum()[df_train.isnull().sum() > 0]

,0


In [11]:
cols_train = set(df_train.columns)
cols_test = set(df_test.columns)

cols_only_in_train = cols_train - cols_test
print("Các cột chỉ có trong train:")
print(cols_only_in_train)

Các cột chỉ có trong train:
{'SalePrice'}


In [12]:
x = df_train.drop(columns=['SalePrice'],axis=1)
y = df_train['SalePrice']


In [13]:
df_train.shape

(1460, 93)

# Pipeline cho encode

In [15]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 1. Xác định loại cột
numeric_cols = x.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = x.select_dtypes(include=['object', 'category']).columns

# 2. Tiền xử lý
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)

# Ridge Regression

In [16]:
# 3. Tạo pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('ridge', Ridge(random_state=42))
])

# 4. Thiết lập lưới tham số (giống cách bạn làm)
param_grid = {
    'ridge__alpha': [0.01, 0.1, 1, 10, 100],   # Mức regularization
    'ridge__solver': ['auto', 'saga', 'lbfgs'] # Solver khác nhau
}

# 5. Dùng K-Fold thay vì Stratified (vì đây là regression)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# 6. Grid Search với nhiều metrics
scoring = {
    'mae': 'neg_mean_absolute_error',
    'rmse': 'neg_root_mean_squared_error',
    'r2': 'r2'
}

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=cv,
    scoring=scoring,
    refit='r2',     # Mô hình tốt nhất theo R²
    n_jobs=-1,
    verbose=1
)

# 7. Train model
print("\nBắt đầu training với GridSearch...")
grid_search.fit(x, y)

# 8. Kết quả
print("\nBest parameters:", grid_search.best_params_)
print("Best cross-validation R²: {:.4f}".format(grid_search.best_score_))

# 9. Kết quả chi tiết từng tham số
cv_results = grid_search.cv_results_
print("\nCross-validation results:")
for i in range(len(cv_results['params'])):
    print(f"Params: {cv_results['params'][i]}")
    print(f"  MAE: {-cv_results['mean_test_mae'][i]:.2f} (+/- {cv_results['std_test_mae'][i]:.2f})")
    print(f"  RMSE: {-cv_results['mean_test_rmse'][i]:.2f} (+/- {cv_results['std_test_rmse'][i]:.2f})")
    print(f"  R²: {cv_results['mean_test_r2'][i]:.4f} (+/- {cv_results['std_test_r2'][i]:.4f})")




Bắt đầu training với GridSearch...
Fitting 5 folds for each of 15 candidates, totalling 75 fits

Best parameters: {'ridge__alpha': 10, 'ridge__solver': 'auto'}
Best cross-validation R²: 0.8393

Cross-validation results:
Params: {'ridge__alpha': 0.01, 'ridge__solver': 'auto'}
  MAE: 20127.90 (+/- 1142.73)
  RMSE: 31844.65 (+/- 3581.92)
  R²: 0.8369 (+/- 0.0306)
Params: {'ridge__alpha': 0.01, 'ridge__solver': 'saga'}
  MAE: nan (+/- nan)
  RMSE: nan (+/- nan)
  R²: nan (+/- nan)
Params: {'ridge__alpha': 0.01, 'ridge__solver': 'lbfgs'}
  MAE: nan (+/- nan)
  RMSE: nan (+/- nan)
  R²: nan (+/- nan)
Params: {'ridge__alpha': 0.1, 'ridge__solver': 'auto'}
  MAE: 20119.16 (+/- 1141.91)
  RMSE: 31838.95 (+/- 3581.01)
  R²: 0.8370 (+/- 0.0306)
Params: {'ridge__alpha': 0.1, 'ridge__solver': 'saga'}
  MAE: nan (+/- nan)
  RMSE: nan (+/- nan)
  R²: nan (+/- nan)
Params: {'ridge__alpha': 0.1, 'ridge__solver': 'lbfgs'}
  MAE: nan (+/- nan)
  RMSE: nan (+/- nan)
  R²: nan (+/- nan)
Params: {'ridge__a

In [17]:
# Mô hình tốt nhất
best_model_ridge = grid_search.best_estimator_

# Dự đoán trên training set
y_pred_train = best_model_ridge.predict(x)

# Tính các metrics
train_mae = mean_absolute_error(y, y_pred_train)
train_rmse = np.sqrt(mean_squared_error(y, y_pred_train))
train_r2 = r2_score(y, y_pred_train)

# In kết quả
print("\n=== KẾT QUẢ TRÊN TRAINING SET ===")
print(f"MAE (Mean Absolute Error): {train_mae:.4f}")
print(f"RMSE (Root Mean Squared Error): {train_rmse:.4f}")
print(f"R² (Coefficient of Determination): {train_r2:.4f}")




=== KẾT QUẢ TRÊN TRAINING SET ===
MAE (Mean Absolute Error): 5674.9239
RMSE (Root Mean Squared Error): 8616.7071
R² (Coefficient of Determination): 0.9882


# Dump mô hình Ridge Regression

In [18]:
# Đường dẫn lưu model
save_path = '/content/drive/MyDrive/house_predict/process/exps/trainbase_31102025'

# Lưu model Ridge tốt nhất
joblib.dump(best_model_ridge, f'{save_path}/ridge_regression_model.pkl')
print("Model Ridge đã được lưu bằng joblib!")

# Lưu thêm thông tin mở rộng
feature_names = x.columns.tolist()

joblib.dump({
    'model': best_model_ridge,
    'feature_names': feature_names,
    'training_date': '2025-10-31'
}, f'{save_path}/ridge_model_package.pkl')

print("Đã lưu đầy đủ model package (model + feature names + ngày huấn luyện).")

# Load lại model để kiểm tra
loaded_package = joblib.load(f'{save_path}/ridge_model_package.pkl')
loaded_model = loaded_package['model']

print("Model đã được load thành công!")

Model Ridge đã được lưu bằng joblib!
Đã lưu đầy đủ model package (model + feature names + ngày huấn luyện).
Model đã được load thành công!


# Lấy Model Train

In [19]:
# Load model đã train
model_package = joblib.load('/content/drive/MyDrive/house_predict/process/exps/trainbase_31102025/ridge_model_package.pkl')
best_model = model_package['model']         # model Ridge đã train
feature_names = model_package['feature_names']


# Chỉ giữ những cột model cần
X_test = df_test[feature_names]

# Dự đoán giá nhà
test_preds = best_model.predict(X_test)

# Tạo submission
submission = pd.DataFrame({
    "Id": df_test["Id"],   # hoặc "HouseId" tùy dataset
    "SalePrice": test_preds
})

# Lưu file submission
submission.to_csv('/content/drive/MyDrive/house_predict/process/exps/trainbase_31102025/submission_ridge.csv', index=False)

# Kiểm tra submission
print("\n=== SUBMISSION INFO ===")
print(f"Shape: {submission.shape}")
print(f"Id range: {submission['Id'].min()} - {submission['Id'].max()}")
print(f"SalePrice stats:\n{submission['SalePrice'].describe()}")



=== SUBMISSION INFO ===
Shape: (1459, 2)
Id range: 1461 - 2919
SalePrice stats:
count      1459.000000
mean     179930.146763
std       73071.837208
min       23161.000645
25%      129522.535168
50%      164141.546445
75%      212193.679616
max      459722.457048
Name: SalePrice, dtype: float64


# RandomForestRegressor

In [23]:
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from sklearn.pipeline import Pipeline

# 1. Chia dữ liệu 80/20
X_train_split, X_valid_split, y_train_split, y_valid_split = train_test_split(
    x, y, test_size=0.2, random_state=42
)

# 2. Pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('rf', RandomForestRegressor(random_state=42))
])

# 3. Lưới tham số (giảm bớt để train nhanh hơn)
param_grid = {
    'rf__n_estimators': [100, 200, 300],
    'rf__max_depth': [None, 10, 20, 30],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4]
}

best_r2 = -np.inf
best_params = None
best_model = None

# 4. Dò lưới tham số
for params in ParameterGrid(param_grid):
    pipeline.set_params(**params)
    pipeline.fit(X_train_split, y_train_split)

    y_pred = pipeline.predict(X_valid_split)
    r2 = r2_score(y_valid_split, y_pred)

    mae = mean_absolute_error(y_valid_split, y_pred)
    rmse = np.sqrt(mean_squared_error(y_valid_split, y_pred))  # fix RMSE

    print(f"Params: {params}")
    print(f"  MAE: {mae:.2f}, RMSE: {rmse:.2f}, R2: {r2:.4f}\n")

    if r2 > best_r2:
        best_r2 = r2
        best_params = params
        best_model = pipeline

print(f"=== BEST MODEL ===")
print(f"Params: {best_params}")
print(f"Validation R2: {best_r2:.4f}")

Params: {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 100}
  MAE: 22691.58, RMSE: 38478.60, R2: 0.8070

Params: {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 200}
  MAE: 22860.23, RMSE: 38483.22, R2: 0.8069

Params: {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 300}
  MAE: 22918.01, RMSE: 38644.69, R2: 0.8053

Params: {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 5, 'rf__n_estimators': 100}
  MAE: 22675.66, RMSE: 38207.37, R2: 0.8097

Params: {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 5, 'rf__n_estimators': 200}
  MAE: 22923.33, RMSE: 38561.24, R2: 0.8061

Params: {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 5, 'rf__n_estimators': 300}
  MAE: 22919.32, RMSE: 38441.41, R2: 0.8073

Params: {'rf__max_depth': None, 'rf__min_samples_leaf': 1,

In [25]:
# Mô hình tốt nhất
best_model_final = best_model

# Dự đoán trên training set
y_pred_train = best_model_ridge.predict(x)

# Tính các metrics
train_mae = mean_absolute_error(y, y_pred_train)
train_rmse = np.sqrt(mean_squared_error(y, y_pred_train))
train_r2 = r2_score(y, y_pred_train)

# In kết quả
print("\n=== KẾT QUẢ TRÊN TRAINING SET ===")
print(f"MAE (Mean Absolute Error): {train_mae:.4f}")
print(f"RMSE (Root Mean Squared Error): {train_rmse:.4f}")
print(f"R² (Coefficient of Determination): {train_r2:.4f}")


=== KẾT QUẢ TRÊN TRAINING SET ===
MAE (Mean Absolute Error): 5674.9239
RMSE (Root Mean Squared Error): 8616.7071
R² (Coefficient of Determination): 0.9882


In [26]:
joblib.dump(best_model_final, f'{save_path}/RandomForestRegressor_model.pkl')
print("Model RandomForest đã được lưu bằng joblib!")

# Lưu thêm thông tin mở rộng
feature_names = x.columns.tolist()

joblib.dump({
    'model': best_model_final,
    'feature_names': feature_names,
    'training_date': '2025-10-31'
}, f'{save_path}/RandomForestRegressor_package.pkl')

# Load lại model để kiểm tra
loaded_package = joblib.load(f'{save_path}/RandomForestRegressor_package.pkl')
loaded_model = loaded_package['model']

print("Model đã được load thành công!")


Model RandomForest đã được lưu bằng joblib!
Model đã được load thành công!


In [27]:
# Load model RandomForest đã train
model_package = joblib.load('/content/drive/MyDrive/house_predict/process/exps/trainbase_31102025/RandomForestRegressor_package.pkl')
best_model = model_package['model']         # model RandomForest đã train
feature_names = model_package['feature_names']

# Chỉ giữ những cột model cần
X_test = df_test[feature_names]

# Dự đoán giá nhà
test_preds = best_model.predict(X_test)

# Tạo submission
submission = pd.DataFrame({
    "Id": df_test["Id"],   # hoặc "HouseId" tùy dataset
    "SalePrice": test_preds
})

# Lưu file submission
submission.to_csv('/content/drive/MyDrive/house_predict/process/exps/trainbase_31102025/submission_randomforest.csv', index=False)

# Kiểm tra submission
print("\n=== SUBMISSION INFO ===")
print(f"Shape: {submission.shape}")
print(f"Id range: {submission['Id'].min()} - {submission['Id'].max()}")
print(f"SalePrice stats:\n{submission['SalePrice'].describe()}")



=== SUBMISSION INFO ===
Shape: (1459, 2)
Id range: 1461 - 2919
SalePrice stats:
count      1459.000000
mean     180721.565784
std       68865.107190
min       69519.326552
25%      133536.205750
50%      162367.312010
75%      205819.305297
max      446038.760226
Name: SalePrice, dtype: float64


# XGBRegressor

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor

X_train_split, X_valid_split, y_train_split, y_valid_split = train_test_split(
    x, y, test_size=0.2, random_state=42
)



# 3. Pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('xgb', XGBRegressor(random_state=42, verbosity=0))
])

# 4. Lưới tham số (giảm bớt để train nhanh)
param_grid = {
    'xgb__n_estimators': [100, 200],
    'xgb__max_depth': [3, 6, 10],
    'xgb__learning_rate': [0.01, 0.1],
    'xgb__subsample': [0.8, 1.0]
}

best_r2 = -np.inf
best_params = None
best_model = None

# 5. Dò lưới tham số
for params in ParameterGrid(param_grid):
    pipeline.set_params(**params)
    pipeline.fit(X_train_split, y_train_split)

    y_pred = pipeline.predict(X_valid_split)
    r2 = r2_score(y_valid_split, y_pred)
    mae = mean_absolute_error(y_valid_split, y_pred)
    rmse = mean_squared_error(y_valid_split, y_pred, squared=False)

    print(f"Params: {params}")
    print(f"  MAE: {mae:.2f}, RMSE: {rmse:.2f}, R2: {r2:.4f}\n")

    if r2 > best_r2:
        best_r2 = r2
        best_params = params
        best_model = pipeline

print(f"=== BEST MODEL ===")
print(f"Params: {best_params}")
print(f"Validation R2: {best_r2:.4f}")